# GENE REPRESENTATIONS FOR DEEP GWAS

## Workspace Setup

### Packages

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import torch as pt

### Global Variables

### Functions

## Data Generation